In [1]:
%reload_ext autoreload
%autoreload 2
import os
os.chdir('/Users/troy/PhD_Research/Deep_Learning_with_Recommendation_system/GCN/char-rnn-tf')
from run_gcn_chars_auxiliary_attention_sample import *

from sqlalchemy import create_engine
import pandas as pd

# get user_id, beer_id, auxiliary

In [2]:
def norm_auxi_value(df):
    for col in ['r_appearance', 'r_aroma', 'r_overall', 'r_palate', 'r_taste']:
        df[col] = df[col] / 5.0
    return df

def load_df(path):
    file = path + '/' + 'beeradvocate.db'

    _file = 'sqlite:///' + file
    engine = create_engine(_file)

    review = engine.execute('SELECT * FROM REVIEWS limit 1')

    df_review = pd.read_sql_table("reviews", engine)
    df_beer = pd.read_sql_table('beers', engine)
    df_total = pd.merge(df_review, df_beer, on='beer_id', how='outer')
    df_total = norm_auxi_value(df_total)

    return df_total

In [3]:
df_total = load_df('/Users/troy/PhD_Research/Deep_Learning_with_Recommendation_system/GCN')

In [4]:
df = pickle.load(open('/Users/troy/PhD_Research/Deep_Learning_with_Recommendation_system/GCN/subsample_5k_py2.pkl', 'rb'))

select dataframe differs between df and df_total

In [5]:
df_validate = df_total.drop(df_total.index[df.index])

filter the validate df user id and item id by df

In [7]:
# user_ids, item_ids = df.user_id.tolist(), df.beer_id.tolist()
# def filte_df_validate(row):
#     if row.user_id in user_ids and row.beer_id in item_ids:
#         return row

# df_val_filter = df_validate.apply(filte_df_validate, axis=1)

In [6]:
df_val_merged = df_validate.drop(df_validate.index[~df_validate.user_id.isin(df.user_id) | ~df_validate.beer_id.isin(df.beer_id)])

In [7]:
len(df_total), len(df), len(df_validate), len(df_val_merged)

(1518467, 5000, 1513467, 146912)

In [8]:
test = df_val_merged.sample()

In [9]:
test

,beer_id,user_id,r_appearance,r_aroma,r_overall,r_palate,r_taste,r_text,r_time,beer_abv,brewer_id,beer_name,beer_style
705499,1670,thekevlarkid,0.8,0.8,0.8,0.8,0.7,off tap this beer was a burnt amber color like...,1.289833e+09,6.5,256,indica india pale ale,american ipa


## user_id, item_id

In [10]:
user_id = test.user_id
item_id = test.beer_id
user_id, item_id

(705499    thekevlarkid
 Name: user_id, dtype: object, 705499    1670
 Name: beer_id, dtype: object)

In [11]:
df[df.user_id.isin(user_id)]

,beer_id,user_id,r_appearance,r_aroma,r_overall,r_palate,r_taste,r_text,r_time,beer_abv,brewer_id,beer_name,beer_style
286871,20564,thekevlarkid,1.0,0.7,0.8,0.7,0.8,out of a bottle this beer was a nearly opaque ...,1.236040e+09,5.8,35,samuel adams holiday porter,american porter
1408156,22851,thekevlarkid,0.8,0.7,0.8,0.7,0.7,off the tap at the rio chama steakhouse this b...,1.231857e+09,5.0,2435,rio chama amber,american amber/red ale
627701,45629,thekevlarkid,0.8,0.8,0.8,0.7,0.8,i have always liked the combination of coffee ...,1.232839e+09,6.8,13884,peak organic espresso amber ale,american amber/red ale
377975,7,thekevlarkid,0.6,0.7,0.8,0.6,0.6,as a former fan of other adulterants how could...,1.220994e+09,4.2,3,purple haze,fruit/vegetable beer


In [12]:
df[df.beer_id.isin(item_id)]

,beer_id,user_id,r_appearance,r_aroma,r_overall,r_palate,r_taste,r_text,r_time,beer_abv,brewer_id,beer_name,beer_style
705053,1670,ricks95,0.7,0.7,0.8,0.8,0.8,"dark bronze in the glass, with a thick cloudy ...",1.251831e+09,6.5,256,indica india pale ale,american ipa
705383,1670,billolick,0.9,0.8,0.8,0.9,0.9,"12 ounce bottle, with no freshness dating. pou...",1.093830e+09,6.5,256,indica india pale ale,american ipa
705112,1670,overlord,0.8,0.4,0.7,0.7,0.7,pours a bright clear orange. inch or two of of...,1.226117e+09,6.5,256,indica india pale ale,american ipa
705278,1670,peatreek,0.8,0.6,0.5,0.6,0.6,"ok, any beer named ""indica"" with a picture of ...",1.148537e+09,6.5,256,indica india pale ale,american ipa


## auxiliary

In [13]:
auxiliary = [float(test.r_appearance), float(test.r_aroma), float(test.r_palate), float(test.r_taste), float(test.r_overall)]

In [14]:
auxiliary

[0.8, 0.8, 0.8, 0.7, 0.8]

## sample numbers, folder path

In [15]:
n_samples = 1000
checkpoint_folder = 'char_tf_one_hot_5k_nb128_bs500_lstm1024_lr0002_gpu'
num_epochs = 451

# samples

In [16]:
sample = generate_sample(n_samples, checkpoint_folder, num_epochs, user_id, item_id, auxiliary)

[34, 53, 54, 52, 34]
('number of auxiliary,', 5)
('number of classes,', 64)
('number of batches,', 128)
('batch size,', 500)
('number of characters per train,', 56)
('number of lstm size,', 1024)
('number of whole characters of this user,', 7168)


In [17]:
sample

u" stored brew with a non-out's creamy color and incredibly coar orange but not dusent. there's some interesting brews not scrent to what i get a redecinarch.  seems a light cherry wheat. tight of musky and sugary. the mouthfeel was a light amount/component. this may be green and complexity. there was a light caramel come through. the mouthfeel was medium-with a semi-bitter feel character.  well, there's more asibit with a generous component. this may be greet and the memories; well balanced. it leaves a light body. the mouthfeel was a light amount/coming. the mouthfeel was over fine, with easy dryness. the mouthfeel was a light american copper. the mouthfeel was a good coaring. when good beer that was an undercenting comparison. the mouthfeel was a good the meat bleering. still, it goes because the nose will continued becoming. the mouthfeel was a good the medicinal, but with a generous compinent. the mouthfeel was a good the medicinal, but with a generous coming through. the mouthfee

In [146]:
results = []
for s in sample.split():
    if s != '_end__str_':
        results.append(s)
    else:
        break
results = ' '.join(results)

for i, c in enumerate(results[::-1]):
    if c == '.':
        break
if i == 0:
    print(results)
else:
    print(results[:-i])

supply smell, but otherw beers. again, more for the beer goes. nothing standous for the review. i don't know if at least. smooth for the style. very nice. amazing malty flavor, with nothing balance. mouthfeel is medium bodied. mouthfeel was mouthfeel, good. impression beer does in the man. maybe i'm definitely recommend pack since they, guess done, with guess good.


In [150]:
results = []
for s in sample.split():
    if s != '_end__str_':
        results.append(s)
    else:
        break
results = ' '.join(results)

for i, c in enumerate(results[::-1]):
    if c == '.':
        break
if i == 0:
    print(results)
else:
    print(results[:-i])

support because however, almost dominate. definitely didn't for what.


In [147]:
print(test.r_text.tolist()[0])

a: dark, hazy brown, nice head, ok retention, good lace.  s: sweet malt  t: very sweet malt, nice touch of caramel, and a slight hop bite. really reminds me of a marzen.  m: slightly sticky, medium body  d: wow, i can drink the hell out of these. good abv, good taste, nicely done.


# generate different samples from different user item pair

In [119]:
num_examples = 100

In [120]:
import itertools

In [121]:
def create_text(sample):
    results = []
    for s in sample.split():
        if s != '_end__str_':
            results.append(s)
        else:
            break
    results = ' '.join(results)

    for i, c in enumerate(results[::-1]):
        if c == '.':
            break
    if i == 0:
        return results
    else:
        return results[:-i]

In [124]:
n_samples = 100
checkpoint_folder = 'char_tf_one_hot_5k_nb128_bs500_lstm1024_lr0002_gpu'
num_epochs = 413
# df = pickle.load(open('/Users/troy/PhD_Research/Deep_Learning_with_Recommendation_system/GCN/subsample_5k_py2.pkl', 'rb'))

uid, iid, auxi, generate_text, original_text = [], [], [], [], []
for i in range(num_examples):
    test = df_val_merged.sample()
    user_id = test.user_id
    item_id = test.beer_id
    
#     for i in range(5):
#         for j in itertools.combinations(range(5), i+1):
#             index = list(j)
#             auxiliary = np.zeros(5)
#             for idx in index:
#                 auxiliary[idx] = 1
                
#             sample = generate_sample(n_samples, checkpoint_folder, num_epochs, user_id, item_id, auxiliary)
    
#             uid.append(list(user_id)[0])
#             iid.append(list(item_id)[0])
#             generate_text.append(sample)
#             original_text.append(list(test.r_text)[0])
#             auxi.append(auxiliary)
            
    
    auxiliary = [float(test.r_appearance), float(test.r_aroma), float(test.r_palate), float(test.r_taste), float(test.r_overall)]
    
    sample = generate_sample(n_samples, checkpoint_folder, num_epochs, user_id, item_id, auxiliary)
    
    uid.append(list(user_id)[0])
    iid.append(list(item_id)[0])
    generate_text.append(create_text(sample))
    original_text.append(list(test.r_text)[0])
    auxi.append(auxiliary)
    
    print(i)

[34, 53, 54, 52, 34]
('number of auxiliary,', 5)
('number of classes,', 64)
('number of batches,', 128)
('batch size,', 500)
('number of characters per train,', 56)
('number of lstm size,', 1024)
('number of whole characters of this user,', 7168)
0
[34, 53, 54, 52, 34]
('number of auxiliary,', 5)
('number of classes,', 64)
('number of batches,', 128)
('batch size,', 500)
('number of characters per train,', 56)
('number of lstm size,', 1024)
('number of whole characters of this user,', 7168)
1
[34, 53, 54, 52, 34]
('number of auxiliary,', 5)
('number of classes,', 64)
('number of batches,', 128)
('batch size,', 500)
('number of characters per train,', 56)
('number of lstm size,', 1024)
('number of whole characters of this user,', 7168)
2
[34, 53, 54, 52, 34]
('number of auxiliary,', 5)
('number of classes,', 64)
('number of batches,', 128)
('batch size,', 500)
('number of characters per train,', 56)
('number of lstm size,', 1024)
('number of whole characters of this user,', 7168)
3
[34,

In [ ]:
# uid, iid, auxi, generate_text, original_text = np.transpose(uid), np.transpose(iid), np.transpose(auxi), np.transpose(generate_text), np.transpose(original_text)

In [125]:
re_df = pd.DataFrame(data=[uid, iid, auxi, generate_text, original_text]).transpose()

In [126]:
re_df.columns = ['user_id', 'item_id', 'auxiliary', 'generate text', 'original text']

In [129]:
re_df

,user_id,item_id,auxiliary,generate text,original text
0,zorprime,48921,"[0.8, 0.8, 0.7, 0.7, 0.6]","not too sweet, cranberries, with hints of grap...",thanks for bobsy for this beer. tasted on sept...
1,morimech,49861,"[0.8, 0.9, 0.8, 0.7, 0.6]",some things more. i think the price to be seem...,pours a clear amber color with one inch of whi...
2,capra12,55180,"[0.7, 0.7, 0.7, 0.5, 0.6]",c,"reviewed out of a a pint glass, appearance: j..."
3,jiggahertz,1005,"[1.0, 0.8, 0.8, 0.8, 0.9]",this is one major leaves hoppiness here. m: we...,a: light translucent amber with a slowly dissi...
4,homebrew311,1769,"[0.9, 1.0, 0.7, 0.8, 0.9]","brewed in a nonic, intriguing seasonal as well.",i was happy to find this beer while on my alas...
5,stcules,412,"[0.8, 0.8, 0.8, 0.8, 0.9]","super dark good, but it has a very dark golden...","the color is absolutely black, opaque. a scarc..."
6,atlas1,1005,"[0.7, 0.7, 0.8, 0.9, 0.8]",... but i didn't know that to me.,"i'm a fan of the victory beers, and this is no..."
7,waltonc,47673,"[0.6, 0.4, 0.6, 0.4, 0.7]",voriboo.,a: pours dark brown with a lot of light tan he...
8,harrymel,39334,"[0.7, 0.9, 0.5, 0.4, 0.5]",review. 12oz bottle pickly sentle from bottle ...,"thanks to jbtice0315 for this as an extra, che..."
9,alewatcher,35608,"[0.8, 0.9, 0.9, 0.9, 1.0]",o,another awesome west coast ipa from an awesome...


In [128]:
re_df.to_csv('sample_results_auxiliary_100.csv')

In [151]:
import numpy as np

In [156]:
l, c = 50, 20

In [154]:
a = np.ones(50)

In [171]:
idx = np.random.randint(low=0, high=0)
idx

ValueError: low >= high